In [ ]:
# ===== Mozilla SCL-BT – Cell 1: setup, imports, paths =====

!pip install -q nltk

import os
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import nltk
nltk.download("punkt", quiet=True)

from google.colab import drive
drive.mount("/content/drive")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Root where your v02 folder lives
DATA_ROOT   = "/content/drive/MyDrive/msr2013-bug_dataset-master/data/v02"
MOZILLA_DIR = f"{DATA_ROOT}/mozilla"

ASSIGNED_FILE   = f"{MOZILLA_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{MOZILLA_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{MOZILLA_DIR}/bug_status.json"
RESOLUTION_FILE = f"{MOZILLA_DIR}/resolution.json"

print("Mozilla folder:", MOZILLA_DIR)
print("Files:")
print(" ", ASSIGNED_FILE)
print(" ", SHORT_DESC_FILE)
print(" ", BUG_STATUS_FILE)
print(" ", RESOLUTION_FILE)


Mounted at /content/drive
Using device: cuda
Mozilla folder: /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla
Files:
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/assigned_to.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/short_desc.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/bug_status.json
  /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/resolution.json


In [ ]:
# ===== Mozilla SCL-BT – Cell 2: build classifier-like DataFrame =====

def load_last_event_table(path, root_key, out_col):
    """
    MSR2013 JSON format:
      { root_key : { bug_id : [ {when, what, who}, ... ], ... } }

    For each bug_id we take the LAST event (max 'when') and keep 'what'.
    Returns DataFrame with columns: ['bug_id', out_col].
    """
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    inner = raw[root_key]
    rows = []

    for bug_id, events in inner.items():
        if not events:
            continue
        last = max(events, key=lambda e: e.get("when", 0))
        rows.append({
            "bug_id": int(bug_id),
            out_col: str(last.get("what", ""))
        })

    return pd.DataFrame(rows)

# 2.1 flatten JSONs
assigned_df   = load_last_event_table(ASSIGNED_FILE,   "assigned_to", "assigned_to")
short_df      = load_last_event_table(SHORT_DESC_FILE, "short_desc",  "short_desc")
bug_status_df = load_last_event_table(BUG_STATUS_FILE, "bug_status",  "bug_status")
resolution_df = load_last_event_table(RESOLUTION_FILE, "resolution",  "resolution")

print("assigned_df head:")
display(assigned_df.head())
print("short_df head:")
display(short_df.head())
print("bug_status_df head:")
display(bug_status_df.head())
print("resolution_df head:")
display(resolution_df.head())

# 2.2 merge on bug_id
df = (assigned_df
      .merge(short_df,      on="bug_id", how="inner")
      .merge(bug_status_df, on="bug_id", how="inner")
      .merge(resolution_df, on="bug_id", how="inner"))

print("Merged rows before filters:", len(df))

# 2.3 keep RESOLVED / VERIFIED / CLOSED AND FIXED
status_upper = df["bug_status"].str.upper()
res_upper    = df["resolution"].str.upper()

mask_status = status_upper.isin(["RESOLVED", "VERIFIED", "CLOSED"])
mask_res    = res_upper.str.contains("FIXED", na=False)

df = df[mask_status & mask_res].copy()
print("Rows after resolved+fixed filter:", len(df))

# 2.4 classifier-style columns (same style as Eclipse)
df["owner"]       = df["assigned_to"].astype(str)
df["issue_title"] = df["short_desc"].astype(str)
df["description"] = ""   # Mozilla files only have short_desc

# keep developers with ≥ MIN_BUGS bugs (you used 2 for DBRNN)
MIN_BUGS = 2
owner_counts = df["owner"].value_counts()
keep_owners  = owner_counts[owner_counts >= MIN_BUGS].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after MIN_BUGS filter:", len(df))
print("Number of developers with ≥2 bugs:", df["owner"].nunique())

# 2.5 encode dev_id
df["dev_id"] = df["owner"].astype("category").cat.codes
n_devs = df["dev_id"].nunique()
print("Final number of developers:", n_devs)

clf_df = df[["owner", "issue_title", "description", "dev_id"]]
display(clf_df.head())


assigned_df head:


bug_id           assigned_to
0     538     terry@mozilla.org
1      91    rickg@netscape.com
2     104  brendan@netscape.com
3     133    ftang@netscape.com
4     174    gagan@netscape.com

short_df head:


bug_id                                         short_desc
0     827            Crashed after adding comments to a bug.
1     818  Internet Explorer can't handle server-push pages.
2     858   [Feature] JavaScript auto-disable per-domain RFE
3     785             {sink} Handling frameset's with bodies
4     672                               yahoo margin problem

bug_status_df head:


bug_id bug_status
0     539   VERIFIED
1     538   VERIFIED
2     540        NEW
3     221   VERIFIED
4     360   VERIFIED

resolution_df head:


bug_id resolution
0     539      FIXED
1     538      FIXED
2     221    INVALID
3     360    INVALID
4     381    INVALID

Merged rows before filters: 394878
Rows after resolved+fixed filter: 115540
Rows after MIN_BUGS filter: 114493
Number of developers with ≥2 bugs: 1843
Final number of developers: 1843


owner                                        issue_title  \
0   terry@mozilla.org   Bugzilla can't search bug lists for a text match   
1  rickg@netscape.com               document properties cannot be listed   
2  ftang@netscape.com  Navigator draws entities like &lt;&amp;&gt; as...   
3  gagan@netscape.com  URL: mailto:'s -> Failed assert on exit at mkg...   
4  ftang@netscape.com  Incorrect display of CP-1250 pages in Unix ver...   

  description  dev_id  
0                1672  
1                1451  
2                 656  
3                 662  
4                 656

In [ ]:
# ===== Mozilla SCL-BT – Cell 3: tokenize + build vocabulary =====

import re
from collections import Counter

# 3.1 combine title + description
clf_df["text"] = (
    clf_df["issue_title"].fillna("").astype(str)
    + " "
    + clf_df["description"].fillna("").astype(str)
)

# 3.2 regex tokenizer (same as Eclipse / Chromium)
token_re = re.compile(r"[a-z]+")

def tokenize(text: str):
    text = str(text).lower()
    return token_re.findall(text)

print("Tokenising all Mozilla bug reports...")
clf_df["tokens"] = clf_df["text"].apply(tokenize)

# 3.3 build frequency vocabulary
MAX_VOCAB = 50000
MIN_FREQ  = 5

freq = Counter()
for toks in clf_df["tokens"]:
    freq.update(toks)

filtered_words = [w for w, c in freq.items() if c >= MIN_FREQ]
filtered_words.sort(key=lambda w: -freq[w])   # by frequency desc
filtered_words = filtered_words[:MAX_VOCAB]

word2id = {w: i for i, w in enumerate(filtered_words)}
id2word = {i: w for w, i in word2id.items()}
vocab_size = len(word2id)

print("Vocab size after cutoff:", vocab_size)

# 3.4 map tokens -> ids
def tokens_to_ids(tokens):
    return [word2id[w] for w in tokens if w in word2id]

clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)

print(clf_df[["text", "tokens", "token_ids"]].head())


/tmp/ipython-input-3830975216.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["text"] = (


Tokenising all Mozilla bug reports...


/tmp/ipython-input-3830975216.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["tokens"] = clf_df["text"].apply(tokenize)


Vocab size after cutoff: 9607
                                                text  \
0  Bugzilla can't search bug lists for a text match    
1              document properties cannot be listed    
2  Navigator draws entities like &lt;&amp;&gt; as...   
3  URL: mailto:'s -> Failed assert on exit at mkg...   
4  Incorrect display of CP-1250 pages in Unix ver...   

                                              tokens  \
0  [bugzilla, can, t, search, bug, lists, for, a,...   
1         [document, properties, cannot, be, listed]   
2  [navigator, draws, entities, like, lt, amp, gt...   
3  [url, mailto, s, failed, assert, on, exit, at,...   
4  [incorrect, display, of, cp, pages, in, unix, ...   

                                           token_ids  
0           [95, 40, 11, 79, 25, 787, 4, 6, 22, 577]  
1                           [91, 208, 156, 17, 2111]  
2  [883, 2897, 1892, 442, 4939, 7143, 5149, 41, 3...  
3    [89, 2149, 36, 330, 545, 3, 845, 43, 114, 1349]  
4               [164,

/tmp/ipython-input-3830975216.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["token_ids"] = clf_df["tokens"].apply(tokens_to_ids)


In [ ]:
# ===== Mozilla SCL-BT – Cell 4: padded sequences + splits =====

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

MAX_LEN = 300   # same as other datasets

sequences = clf_df["token_ids"].tolist()
X_all = pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

y_all = clf_df["dev_id"].values.astype("int32")

print("Number of developers:", n_devs)
print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

bug_indices = np.arange(len(clf_df))

# 80% train, 20% test
train_idx, test_idx, y_train_tmp, y_test = train_test_split(
    bug_indices,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# from train part, 10% as validation
train_idx, val_idx, y_train, y_val = train_test_split(
    train_idx,
    y_train_tmp,
    test_size=0.1,
    random_state=42,
    stratify=y_train_tmp,
)

print(f"Train bugs: {len(train_idx)}  Val bugs: {len(val_idx)}  Test bugs: {len(test_idx)}")

X_train = X_all[train_idx]
X_val   = X_all[val_idx]
X_test  = X_all[test_idx]

y_train = y_all[train_idx]
y_val   = y_all[val_idx]
y_test  = y_all[test_idx]

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)


Number of developers: 1843
X_all shape: (114493, 300)
y_all shape: (114493,)
Train bugs: 82434  Val bugs: 9160  Test bugs: 22899
Train: (82434, 300)  Val: (9160, 300)  Test: (22899, 300)


In [ ]:
# ===== Mozilla SCL-BT – Cell 5: Torch datasets & dataloaders =====

from torch.utils.data import Dataset, DataLoader

BATCH_SIZE = 256

class BugDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BugDataset(X_train, y_train)
val_dataset   = BugDataset(X_val,   y_val)
test_dataset  = BugDataset(X_test,  y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  drop_last=False)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print("Loaders ready.")


Loaders ready.


In [ ]:
# ===== Mozilla SCL-BT – Cell 6: model definition =====

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

EMBED_DIM  = 128
HIDDEN_DIM = 128
N_CLASSES  = n_devs        # from Cell 2
VOCAB_SIZE = vocab_size    # from Cell 3

class BugEncoder(nn.Module):
    """
    Embedding -> BiLSTM -> attention pooling -> bug representation z_b
    """
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            emb_dim,
            hidden_dim,
            batch_first=True,
            bidirectional=True,
        )
        self.attn = nn.Linear(2 * hidden_dim, 1)

    def forward(self, x):
        emb = self.embedding(x)            # [B, T, E]
        h, _ = self.lstm(emb)             # [B, T, 2H]

        scores = torch.tanh(self.attn(h)).squeeze(-1)   # [B, T]
        mask = (x != 0)                                  # [B, T]
        scores = scores.masked_fill(~mask, -1e9)

        alpha = torch.softmax(scores, dim=1)             # [B, T]
        z = torch.sum(h * alpha.unsqueeze(-1), dim=1)    # [B, 2H]
        return z


class SCLBTModel(nn.Module):
    """
    SCL-BT style model for Mozilla.
    """
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.encoder = BugEncoder(vocab_size, emb_dim, hidden_dim)
        rep_dim = 2 * hidden_dim

        self.dev_embed = nn.Embedding(n_classes, rep_dim)  # kept for consistency
        self.classifier = nn.Linear(rep_dim, n_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return logits, z

model = SCLBTModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, N_CLASSES).to(device)
print(model)


Using device: cuda
SCLBTModel(
  (encoder): BugEncoder(
    (embedding): Embedding(9607, 128, padding_idx=0)
    (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
    (attn): Linear(in_features=256, out_features=1, bias=True)
  )
  (dev_embed): Embedding(1843, 256)
  (classifier): Linear(in_features=256, out_features=1843, bias=True)
)


In [ ]:
# ===== Mozilla SCL-BT – Cell 7: SCL loss + training =====

def supervised_contrastive_loss(features, labels, temperature=0.1):
    """
    features: [B, D]
    labels  : [B]
    """
    device_ = features.device
    labels = labels.contiguous().view(-1, 1)   # [B,1]
    batch_size = features.shape[0]

    mask = torch.eq(labels, labels.T).float().to(device_)
    logits_mask = torch.ones_like(mask) - torch.eye(batch_size, device=device_)
    mask = mask * logits_mask

    feats = F.normalize(features, dim=1)
    logits = torch.div(torch.matmul(feats, feats.T), temperature)

    logits_max, _ = torch.max(logits, dim=1, keepdim=True)
    logits = logits - logits_max.detach()

    exp_logits = torch.exp(logits) * logits_mask
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True) + 1e-8)

    mask_sum = mask.sum(1)
    mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)

    loss = -(mask * log_prob).sum(1) / mask_sum
    loss = loss.mean()
    return loss

LAMBDA_SCL = 0.2
EPOCHS     = 5
LR         = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
best_val_loss = float("inf")
best_state = None

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    total_ce   = 0.0
    total_scl  = 0.0
    total_cnt  = 0

    with torch.set_grad_enabled(train):
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)

            logits, z = model(xb)
            loss_ce  = F.cross_entropy(logits, yb)
            loss_scl = supervised_contrastive_loss(z, yb)

            loss = loss_ce + LAMBDA_SCL * loss_scl

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            bs = xb.size(0)
            total_loss += loss.item() * bs
            total_ce   += loss_ce.item() * bs
            total_scl  += loss_scl.item() * bs
            total_cnt  += bs

    return (
        total_loss / total_cnt,
        total_ce   / total_cnt,
        total_scl  / total_cnt,
    )

for epoch in range(1, EPOCHS + 1):
    train_loss, train_ce, train_scl = run_epoch(train_loader, train=True)
    val_loss,   val_ce,   val_scl   = run_epoch(val_loader,   train=False)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} (CE={train_ce:.4f}, SCL={train_scl:.4f})  "
        f"val_loss={val_loss:.4f} (CE={val_ce:.4f}, SCL={val_scl:.4f})"
    )

    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device)

print("Training finished. Best val loss:", best_val_loss)


Epoch 01 | train_loss=5.3253 (CE=4.6410, SCL=3.4214)  val_loss=4.9057 (CE=4.2276, SCL=3.3904)
Epoch 02 | train_loss=4.7892 (CE=4.1062, SCL=3.4147)  val_loss=4.6250 (CE=3.9436, SCL=3.4069)
Epoch 03 | train_loss=4.4488 (CE=3.7652, SCL=3.4177)  val_loss=4.4480 (CE=3.7671, SCL=3.4045)
Epoch 04 | train_loss=4.1517 (CE=3.4727, SCL=3.3946)  val_loss=4.3502 (CE=3.6686, SCL=3.4079)
Epoch 05 | train_loss=3.9002 (CE=3.2242, SCL=3.3801)  val_loss=4.2967 (CE=3.6161, SCL=3.4029)
Training finished. Best val loss: 4.296693046020108


In [ ]:
# ===== Mozilla SCL-BT – Cell 8: evaluation Hit@K =====

model.eval()
all_logits = []
all_labels = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        logits, _ = model(xb)
        all_logits.append(logits.cpu())
        all_labels.append(yb.cpu())

all_logits = torch.cat(all_logits, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()

probs = torch.softmax(torch.from_numpy(all_logits), dim=1).numpy()

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

TOPKS = [1, 3, 5, 7, 9]
hrs = []

print("=== SCL-BT-style on Mozilla (bug encoder + supervised contrastive) ===")
for k in TOPKS:
    hr = hit_at_k(probs, all_labels, k)
    hrs.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr = float(np.mean(hrs))
print(f"Average Hit Ratio SCL-BT-style (Mozilla) : {avg_hr:.4f}")


=== SCL-BT-style on Mozilla (bug encoder + supervised contrastive) ===
Top-1 Hit Ratio: 0.4000
Top-3 Hit Ratio: 0.5175
Top-5 Hit Ratio: 0.5621
Top-7 Hit Ratio: 0.5909
Top-9 Hit Ratio: 0.6116
Average Hit Ratio SCL-BT-style (Mozilla) : 0.5364
